In [40]:
import pandas as pd
import dask.dataframe as dd

In [41]:
d = [
    {
        'id':[2],
        'object_names':['bab'],
        'paths':['/bab'],
        'size':[12],
        'type':['zip'],
        'upload':[True]
    },
    {
        'id':[1],
        'object_names':['bob'],
        'paths':['/bob'],
        'size':[2],
        'type':['zip'],
        'upload':[False]
    },
    {
        'id':[0],
        'object_names':['bib'],
        'paths':['/bib'],
        'size':[9],
        'type':['file'],
        'upload':[True]
        }
]

In [42]:
ddf = dd.from_pandas(pd.DataFrame([], columns=['id', 'object_names', 'paths', 'size', 'type', 'upload']))

In [43]:
for de in d:
    df = pd.DataFrame(de)
    if (len(ddf.index) == 0):
        ddf = dd.from_pandas(df, npartitions=1)
    else:
        ddf = dd.concat([ddf, dd.from_pandas(df, npartitions=1)])

In [44]:
ddf

,id,object_names,paths,size,type,upload
npartitions=3,,,,,,
,int64,string,string,int64,string,bool
,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [45]:
del d[2]

In [46]:
d

[{'id': [2],
  'object_names': ['bab'],
  'paths': ['/bab'],
  'size': [12],
  'type': ['zip'],
  'upload': [True]},
 {'id': [1],
  'object_names': ['bob'],
  'paths': ['/bob'],
  'size': [2],
  'type': ['zip'],
  'upload': [False]}]

In [47]:
ddf.compute()

,id,object_names,paths,size,type,upload
0,2,bab,/bab,12,zip,True
0,1,bob,/bob,2,zip,False
0,0,bib,/bib,9,file,True


In [48]:
import os

In [49]:
path = os.path.abspath('../../../test_tree_large')

In [50]:
path

'/home/dave/test_tree_large'

In [51]:
df = pd.DataFrame()
folder_count = 0
file_count = 0
for folder, _, files in os.walk(path, topdown=True):
    if len(files) > 0:
        folder_count += 1
        file_count += len(files)
        # print(f'folder: {folder}')
        # print(f'sub_folders: {sub_folders}')
        # print(f'files: {files}')
        folder_files = [os.path.join(folder, file) for file in files]
        df = pd.concat([df, pd.DataFrame(folder_files, columns=['local_path'])], ignore_index=True)

In [52]:
df['size'] = df['local_path'].apply(lambda x: os.path.getsize(x))
df['object_names'] = df['local_path'].apply(lambda x: os.path.basename(x))
df['upload'] = False
df['type'] = 'file'

In [53]:
ddf = dd.from_pandas(df, npartitions=100)
ddf

,local_path,size,object_names,upload,type
npartitions=100,,,,,
0,string,int64,string,bool,string
91,...,...,...,...,...
...,...,...,...,...,...
8913,...,...,...,...,...
9002,...,...,...,...,...


In [54]:
current_objects = pd.Series(os.listdir('../../../test_tree_large/7/'))

In [55]:
current_objects = pd.DataFrame(current_objects, columns=['object_names'])
current_objects

,object_names
0,7_256.f
1,7_175.f
2,7_609.f
3,7_769.f
4,7_422.f
...,...
996,7_834.f
997,7_845.f
998,7_195.f
999,7_507.f


In [56]:
ddf['exists'] = ddf['object_names'].isin(current_objects['object_names'])

In [57]:
done = ddf.compute()

In [58]:
uploads = done[done['exists'] == False].drop(columns=['exists'])

In [59]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [60]:
max_zip_size = 1048576 * 100

In [61]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [62]:
import numpy as np

In [63]:

uploads['individual_upload'] = np.random.choice([True, False], size=len(uploads))
uploads

,local_path,size,object_names,upload,type,individual_upload
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,True
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,True
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,False
...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,True
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,True
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False


In [64]:
len(uploads)

8003

In [65]:

cumulative_size = 0
batches = [None]
batch = [None]
batch_number = [None]
for row in uploads.iterrows():
    if row[1]['individual_upload']:
        batch_number.append(0)
    else:
        size = row[1]['size']
        if cumulative_size + size > max_zip_size:
            batches.append(1)
            batch = [1]
            cumulative_size = size
        else:
            batch.append(1)
            cumulative_size += size
            print(row[1].name)
            if row[1].name == uploads.index[-1]:
                print('inif')
                batches.append(1)
        batch_number.append(len(batches))
    # print(f'row: {row[1].name} - batch: {len(batches)} - cumulative_size: {cumulative_size} - size: {size} - individual_upload: {row[1]["individual_upload"]}')
    # k+=1
    # if k == 10:
        # break

batch_number[-10:]

1
3
4
6
8
9
11
12
15
16
21
22
25
26
28
33
35
36
42
43
44
47
57
59
60
61
64
65
72
73
74
75
77
78
79
80
81
84
86
87
88
89
90
92
93
94
96
100
103
104
107
109
112
114
115
116
118
120
121
122
123
124
126
127
134
135
137
139
140
141
142
144
146
148
153
156
170
174
175
176
177
178
181
182
184
186
187
189
193
196
197
200
202
203
205
207
210
218
221
222
224
225
226
228
230
231
232
233
236
238
245
247
249
250
253
254
255
256
257
260
265
267
275
277
279
280
283
284
285
287
291
292
295
296
297
300
303
305
306
308
311
312
315
316
319
321
322
326
332
333
334
336
338
339
343
344
346
347
348
350
352
358
359
360
361
363
364
367
368
369
370
371
372
378
379
380
381
382
384
386
387
389
392
393
394
396
397
399
403
405
406
410
411
412
413
414
415
417
422
427
428
431
432
434
435
437
440
441
442
443
444
445
446
447
448
449
451
456
458
460
461
462
463
464
466
467
470
474
476
478
479
480
487
488
489
491
494
495
496
497
503
504
506
507
510
512
515
516
517
518
523
526
528
530
531
532
534
536
538
541
543
544
545
5

[0, 40, 40, 0, 0, 0, 0, 0, 40, 41]

In [66]:
len(batch_number)

8004

In [67]:
uploads['batch_number'] = batch_number[1:]
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,True,0
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,True,0
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False,1
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,False,1
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,True,0
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,True,0
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [68]:
uploads[uploads['batch_number'] == 1]['size'].sum()

103809300

In [69]:
for bn in range(1, uploads['batch_number'].max()+1):
    size = uploads[uploads['batch_number'] == bn]['size'].sum()
    count = len(uploads[uploads['batch_number'] == bn])
    print(f'Batch Number: {bn}, Size: {size}, Count: {count}')
    if size > max_zip_size:
        print('Size is greater than max zip size')
    else:
        print('Size is less than max zip size')

Batch Number: 1, Size: 103809300, Count: 100
Size is less than max zip size
Batch Number: 2, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 3, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 4, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 5, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 6, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 7, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 8, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 9, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 10, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 11, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 12, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 13, Size: 104857600, Count: 100
Size is less than max zip size
Batch Nu

In [70]:
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,True,0
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,True,0
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False,1
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,False,1
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,True,0
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,True,0
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [71]:
def list_aggredator(x):
    return '|'.join(x)

In [72]:
zips = uploads[uploads['batch_number'] > 0][['batch_number']].drop_duplicates()

In [73]:
zips['path'] = uploads[uploads['batch_number'] > 0]['local_path'].groupby(uploads['batch_number']).agg(list_aggredator).values
zips['size'] = uploads[uploads['batch_number'] > 0]['size'].groupby(uploads['batch_number']).sum().values
zips['object_names'] = uploads[uploads['batch_number'] > 0]['object_names'].groupby(uploads['batch_number']).agg(list_aggredator).values
zips['upload'] = True
zips['uploaded'] = False
zips['type'] = 'zip'
zips['id'] = zips['batch_number']
zips = zips.drop(columns=['batch_number'])
zips

,path,size,object_names,upload,uploaded,type,id
1,/home/dave/test_tree_large/test_tree_traversal...,103809300,test_tree_traversal.yaml|3_425.f|3_805.f|3_238...,True,False,zip,1
223,/home/dave/test_tree_large/3/3_274.f|/home/dav...,104857600,3_274.f|3_411.f|3_905.f|3_5.f|3_287.f|3_463.f|...,True,False,zip,2
425,/home/dave/test_tree_large/3/3_559.f|/home/dav...,104857600,3_559.f|3_51.f|3_740.f|3_233.f|3_823.f|3_942.f...,True,False,zip,3
634,/home/dave/test_tree_large/3/3_296.f|/home/dav...,104857600,3_296.f|3_27.f|3_676.f|3_150.f|3_85.f|3_370.f|...,True,False,zip,4
812,/home/dave/test_tree_large/3/3_558.f|/home/dav...,104857600,3_558.f|3_854.f|3_847.f|3_934.f|3_346.f|3_245....,True,False,zip,5
998,/home/dave/test_tree_large/3/3_421.f|/home/dav...,104857600,3_421.f|3_956.f|5_944.f|5_735.f|5_969.f|5_630....,True,False,zip,6
2214,/home/dave/test_tree_large/5/5_616.f|/home/dav...,104857600,5_616.f|5_730.f|5_417.f|5_27.f|5_793.f|5_731.f...,True,False,zip,7
2405,/home/dave/test_tree_large/5/5_66.f|/home/dave...,104857600,5_66.f|5_403.f|5_440.f|5_268.f|5_815.f|5_152.f...,True,False,zip,8
2627,/home/dave/test_tree_large/5/5_94.f|/home/dave...,104857600,5_94.f|5_37.f|5_190.f|5_608.f|5_864.f|5_705.f|...,True,False,zip,9
2847,/home/dave/test_tree_large/5/5_578.f|/home/dav...,104857600,5_578.f|5_841.f|5_644.f|5_902.f|5_748.f|5_540....,True,False,zip,10


In [74]:
ind_files = uploads[uploads['batch_number'] == 0].drop(columns=['batch_number', 'individual_upload'])
ind_files['upload'] = True
ind_files['uploaded'] = False
ind_files.rename(columns={'local_path': 'path'}, inplace=True)
ind_files['id'] = None
ind_files

,path,size,object_names,upload,type,uploaded,id
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,True,file,False,None
2,/home/dave/test_tree_large/test.csv,334208,test.csv,True,file,False,None
5,/home/dave/test_tree_large/3/3_795.f,1048576,3_795.f,True,file,False,None
7,/home/dave/test_tree_large/3/3_562.f,1048576,3_562.f,True,file,False,None
10,/home/dave/test_tree_large/3/3_802.f,1048576,3_802.f,True,file,False,None
...,...,...,...,...,...,...,...
8996,/home/dave/test_tree_large/1/1_314.f,1048576,1_314.f,True,file,False,None
8997,/home/dave/test_tree_large/1/1_654.f,1048576,1_654.f,True,file,False,None
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,True,file,False,None
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,True,file,False,None


In [75]:
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,True,0
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,True,0
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,False,1
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,False,1
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,True,0
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,True,0
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [76]:
from dask import dataframe as dd

In [262]:
ddf = dd.from_pandas(uploads, 2)

In [263]:
ddf

,local_path,size,object_names,upload,type,individual_upload,batch_number
npartitions=2,,,,,,,
0,string,int64,string,bool,string,bool,int64
5002,...,...,...,...,...,...,...
9002,...,...,...,...,...,...,...


In [95]:
def dd_list_aggregator(x):
    return '|'.join(x)

In [229]:
dd_list_aggregation = dd.Aggregation(
    'list_aggregation',
    list,  # chunks are aggregated into a list of strings
    list,  # strings are aggregated into a list of strings
    lambda x: '|'.join(x) #, meta=('x', 'object'),
    # '|'.join        # finalize by joining the list of strings
)

In [264]:
# ddf2 = dd.from_pandas(pd.DataFrame.from_dict({'local_path':[], 'batch_number':[]}), 2)
ddf2 = dd.from_pandas(pd.DataFrame.from_dict(
    {
        'local_path': [],
        'batch_number': []
    },
))

In [265]:
ddf2['batch_number'] = ddf['batch_number'].drop_duplicates()
# ddf2 = ddf2.compute()

In [266]:
ddf2['batch_number'] = ddf2['batch_number'].astype('int')
ddf2

,local_path,batch_number
npartitions=2,,
,float64,int64
,...,...
,...,...


In [267]:
# ddf[ddf['batch_number'] == 1][['local_path', 'batch_number']].groupby('batch_number').agg(list)['local_path'].compute().iloc[0]

In [268]:
ddf2

,local_path,batch_number
npartitions=2,,
,float64,int64
,...,...
,...,...


In [ ]:
#  = ddf[['batch_number','local_path']].groupby(ddf['batch_number'])['local_path'].agg(list)
ddf2['local_path'] = ddf['local_path'].groupby(ddf['batch_number']).agg(list)
# ddf2.compute()
# a = a.apply().compute()
ddf2['local_path'] = ddf2['local_path'].apply(lambda x: '|'.join(x), meta=('x', 'object'))

In [271]:
ddf2.compute()

TypeError: can only join an iterable